<a href="https://colab.research.google.com/github/LuisaPolicarpo/Project3/blob/main/Marta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# title.akas.tsv.gz - Contains the following information for titles:
# titleId (string) - a tconst, an alphanumeric unique identifier of the title
# ordering (integer) – a number to uniquely identify rows for a given titleId
# title (string) – the localized title
# region (string) - the region for this version of the title
# language (string) - the language of the title
# types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
# attributes (array) - Additional terms to describe this alternative title, not enumerated
# isOriginalTitle (boolean) – 0: not original title; 1: original title

# Marta
# title.basics.tsv.gz - Contains the following information for titles:
# tconst (string) - alphanumeric unique identifier of the title
# titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
# primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
# originalTitle (string) - original title, in the original language
# isAdult (boolean) - 0: non-adult title; 1: adult title
# startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
# endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
# runtimeMinutes – primary runtime of the title, in minutes
# genres (string array) – includes up to three genres associated with the title

# Marta
# title.crew.tsv.gz – Contains the director and writer information for all the titles in IMDb. Fields include:
# tconst (string) - alphanumeric unique identifier of the title
# directors (array of nconsts) - director(s) of the given title
# writers (array of nconsts) – writer(s) of the given title

# title.episode.tsv.gz – Contains the tv episode information. Fields include:
# tconst (string) - alphanumeric identifier of episode
# parentTconst (string) - alphanumeric identifier of the parent TV Series
# seasonNumber (integer) – season number the episode belongs to
# episodeNumber (integer) – episode number of the tconst in the TV series

# title.principals.tsv.gz – Contains the principal cast/crew for titles
# tconst (string) - alphanumeric unique identifier of the title
# ordering (integer) – a number to uniquely identify rows for a given titleId
# nconst (string) - alphanumeric unique identifier of the name/person
# category (string) - the category of job that person was in
# job (string) - the specific job title if applicable, else '\N'
# characters (string) - the name of the character played if applicable, else '\N'

#self-category: 
# Appear in titles with the genre Documentary
# Appear in titles with the keyword TV-special (e.g. the Academy Awards ceremony broadcast)
# Appear in titles with the keyword non-fiction (e.g. Talk-Shows)
# Are credited as "himself, herself, self, themselves" (e.g. in Making-of featurettes or Reality TV)
# are displayed in a separate section called "Self" and other roles will appear in their respective groupings.

# title.ratings.tsv.gz – Contains the IMDb rating and votes information for titles
# tconst (string) - alphanumeric unique identifier of the title
# averageRating – weighted average of all the individual user ratings
# numVotes - number of votes the title has received

# name.basics.tsv.gz – Contains the following information for names:
# nconst (string) - alphanumeric unique identifier of the name/person
# primaryName (string)– name by which the person is most often credited
# birthYear – in YYYY format
# deathYear – in YYYY format if applicable, else '\N'
# primaryProfession (array of strings)– the top-3 professions of the person
# knownForTitles (array of tconsts) – titles the person is known for

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
#IMDB_name.basics
pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/title.basics.tsv.gz', compression = 'gzip', sep = '\t')

In [10]:
#IMDB_title.basics
title_basics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
title_basics

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,\N
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,\N
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,\N
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,\N
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,\N
...,...,...,...,...,...,...,...,...,...
9301897,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,\N
9301898,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,\N
9301899,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,\N
9301900,tt9916856,short,The Wind,The Wind,0,2015,\N,27,\N


In [ ]:
#IMDB_title.akas
pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/title.episode.tsv.gz', compression = 'gzip', sep = '\t')

In [6]:
#IMDB_title.crew
title_crew = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/title.crew.tsv.gz', compression = 'gzip', sep = '\t')
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
9301897,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9301898,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9301899,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9301900,tt9916856,nm10538645,nm6951431


In [ ]:
#IMDB_title.episode
pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/title.episode.tsv.gz', compression = 'gzip', sep = '\t')

In [ ]:
#IMDB_title.principals
pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/title.principals.tsv.gz', compression = 'gzip', sep = '\t')

In [ ]:
#IMDB_title.ratings
pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/title.ratings.tsv.gz', compression = 'gzip', sep = '\t')

## Know the data

# M1) title_basics

In [11]:
#Check the types of variables and null values
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9301902 entries, 0 to 9301901
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 638.7+ MB


In [ ]:
# object ok = tconst, titleTyoe, primaryTitle, originalTitle

# isAdult: is bool, check if there are only 0 and 1

# runtimeMinutes : should be int

In [19]:
#check which variables the isAdult column has
title_basics['isAdult'].unique()

array([0, 1, 2019, 1981, 2020, 2017, '0', '1', '\\N', 2014, 2005],
      dtype=object)

In [43]:
#check which variables the isAdult column has and count them
title_basics['isAdult'].value_counts()

0       8949785
1        286572
0         63380
1          2155
2019          3
2020          2
1981          1
2017          1
\N            1
2014          1
2005          1
Name: isAdult, dtype: int64

In [ ]:
#countercheck
condition = title_basics['isAdult'] == 2019
condition.unique()

In [30]:
# go to imbd site and check what happens in theses tv episodes: 
condition = title_basics['isAdult'] == 2019
title_basics.loc[condition]

# Answer:
# it's the year when they aired the episode for 2020, 2019, 1981, 2017, 2014, 2005

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1097688,tt10233364,tvEpisode,Rolling in the Deep Dish\tRolling in the Deep ...,0,2019,\N,\N,\N,NaN
1509018,tt10970874,tvEpisode,Die Bauhaus-Stadt Tel Aviv - Vorbild für die M...,0,2019,\N,\N,\N,NaN
9258502,tt9822816,tvEpisode,Zwischen Vertuschung und Aufklärung - Missbrau...,0,2019,\N,\N,\N,NaN


In [32]:
#check if other tv episodes show the date as start year or if they are //N 
condition = title_basics['titleType'] == 'tvEpisode'
title_basics.loc[condition]

#Answer: yes it is

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
20346,tt0020666,tvEpisode,Barnacle Bill,Barnacle Bill,0,1930,\N,8,\N
20507,tt0020829,tvEpisode,Dizzy Dishes,Dizzy Dishes,0,1930,\N,6,\N
20832,tt0021166,tvEpisode,Mysterious Mose,Mysterious Mose,0,1930,\N,6,\N
21262,tt0021612,tvEpisode,Any Little Girl That's a Nice Little Girl,Any Little Girl That's a Nice Little Girl,0,1931,\N,7,\N
21305,tt0021655,tvEpisode,Betty Co-ed,Betty Co-ed,0,1931,\N,6,\N
...,...,...,...,...,...,...,...,...,...
9301896,tt9916846,tvEpisode,Episode #3.18,Episode #3.18,0,2010,\N,\N,\N
9301897,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,\N
9301898,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,\N
9301899,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,\N


In [ ]:
#TODO replace the years checked in the startYear column

In [35]:
condition = title_basics['isAdult'] == 0 | 1
title_basics.loc[condition]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
59139,tt0060313,movie,"Dingle, Dangle","Dingle, Dangle",1,1966,\N,61,\N
60703,tt0061926,movie,Lust Weekend,Lust Weekend,1,1967,\N,70,\N
61119,tt0062361,movie,Thigh Spy,Thigh Spy,1,1967,\N,70,\N
61174,tt0062417,movie,Un épais manteau de sang,Un épais manteau de sang,1,1968,\N,88,\N
61478,tt0062727,short,Of Special Merit,Besonders wertvoll,1,1968,\N,11,\N
...,...,...,...,...,...,...,...,...,...
9301376,tt9915710,video,Sodomizer 5,Sodomizer 5,1,1996,\N,85,\N
9301401,tt9915772,tvEpisode,Cassidy Banks loves the Hitachi Wand,Cassidy Banks loves the Hitachi Wand,1,\N,\N,\N,\N
9301405,tt9915788,tvEpisode,Teen travel mates hot fucking,Teen travel mates hot fucking,1,\N,\N,\N,\N
9301614,tt9916254,video,Big Tit Cream Pie 32,Big Tit Cream Pie 32,1,2015,\N,226,\N


In [38]:
condition = title_basics['isAdult'] == '0'
title_basics.loc[condition]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
2949120,tt13607682,tvEpisode,Episode #1.973,Episode #1.973,0,\N,\N,\N,\N
2949121,tt13607684,tvEpisode,Episode #1.975,Episode #1.975,0,\N,\N,\N,\N
2949122,tt13607686,tvEpisode,Episode #1.976,Episode #1.976,0,\N,\N,\N,\N
2949123,tt13607688,tvEpisode,Episode #1.977,Episode #1.977,0,\N,\N,\N,\N
2949124,tt1360768,movie,David Farrell: Elusive Moments,David Farrell: Elusive Moments,0,2008,\N,65,\N
...,...,...,...,...,...,...,...,...,...
3014651,tt13729658,tvEpisode,Vvedenie,Vvedenie,0,2021,\N,54,\N
3014652,tt1372965,tvEpisode,Episode dated 3 February 2009,Episode dated 3 February 2009,0,2009,\N,\N,\N
3014653,tt13729660,tvEpisode,Karma,Karma,0,2021,\N,\N,\N
3014654,tt13729662,tvMovie,Kennzeichen DAH - Kainszeichen Dachau,Kennzeichen DAH - Kainszeichen Dachau,0,1971,\N,45,\N


In [53]:
#check value_counts other columns
#total entries: 9301902

title_basics['tconst'].nunique()
#9301902

9301902

In [54]:
title_basics['titleType'].value_counts()
#11 unique values
#no /N values

tvEpisode       7018334
short            894908
movie            624333
video            266185
tvSeries         232702
tvMovie          137978
tvMiniSeries      45579
tvSpecial         38677
videoGame         32504
tvShort           10700
tvPilot               2
Name: titleType, dtype: int64

In [55]:
title_basics['primaryTitle'].nunique()
#4262092 unique values

title_basics['primaryTitle'].nunique()

4262092

In [ ]:
# # on isAdult column - change all other than 0 and 1 to null

# def check_safety(rating):
#   safe_for_children = ''
#   if rating == 'G':
#     safe_for_children = 'yes'
#   else:
#     safe_for_children = 'no' 
#   return safe_for_children

# movies['safe_for_children'] = movies['content_rating'].apply(check_safety)
# movies.head()


# M2) Tile crew

In [9]:
title_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9301902 entries, 0 to 9301901
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   tconst     object
 1   directors  object
 2   writers    object
dtypes: object(3)
memory usage: 212.9+ MB
